In [2]:
import pandas as pd
from scipy.stats import ttest_rel,probplot
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
#import math
#import copy as cp
#from collections import defaultdict


#from surprise
from surprise import KNNWithMeans,SVD,SVDpp,NMF,SlopeOne,CoClustering
from surprise import Reader, Dataset,accuracy
from surprise.model_selection import cross_validate, KFold, ShuffleSplit, GridSearchCV
from surprise.prediction_algorithms import PredictionImpossible
from surprise.model_selection import train_test_split as surprisesplit
from surprise import dump

#user defined functions
#from prec_recall import precision_recall_at_k,pr_eval
#from sigweight import KNNSigWeighting
#from Rank import *
#from data_cleaning import *
#from hybrid import WeightedHybrid

In [3]:
df_rating = pd.read_csv(r'archive (2)\rating.csv')
df_anime = pd.read_csv(r'archive (2)\anime.csv')

In [4]:
#drop rows with no rating (-1)
mask = df_rating['rating']!=-1
df_rating = df_rating[mask]

In [5]:
df_anime

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [6]:
from data_cleaning import *

In [7]:
#10-Core Operation & One-Scale Rater Elimination on Anime_rating using clean_data()
core = data_cleaner(df_rating)

## Iteration 1 ##
There are one-scale raters!
one_Scale_rater_eliminator excuted


There are some user or item (if not None) has less than 10 ratings!
core_operator excuted!


## Iteration 2 ##
No one scale Rater!


No item & user has less than 10 ratinngs!


Data is cleaned!


In [8]:
core.to_csv('core_rating.csv',index=False)

In [9]:
print(f"Number of operations {len(core)}")

for i in core.nunique().index:
    print(f"Number of unique {i.split('_')[0]}s: {core.nunique()[i]}")

print(f'*rating scales: {np.sort(core['rating'].unique())}')

print(f'Density: {len(core)/(core.nunique()[0]*core.nunique().iloc[1])}')

core.head(5)


Number of operations 6247010
Number of unique users: 54743
Number of unique animes: 7359
Number of unique ratings: 10
*rating scales: [ 1  2  3  4  5  6  7  8  9 10]


C:\Users\diyas\AppData\Local\Temp\ipykernel_21708\3568393601.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Density: {len(core)/(core.nunique()[0]*core.nunique().iloc[1])}')


Density: 0.015506893510584463


,user_id,anime_id,rating
156,3,20,8
157,3,154,6
158,3,170,9
159,3,199,10
160,3,225,9


In [10]:
#limit the experiment to tv comedy 
#limit user data to tv & comedy based only
tv_comedy_anime= Anime_subset(df_anime,df_rating,'TV','Comedy')

#perform eliminaton & core_operation
tv_comedy_anime_core=data_cleaner(tv_comedy_anime)

## Iteration 1 ##
There are one-scale raters!
one_Scale_rater_eliminator excuted


There are some user or item (if not None) has less than 10 ratings!
core_operator excuted!


## Iteration 2 ##
No one scale Rater!


There are some user or item (if not None) has less than 10 ratings!
core_operator excuted!


## Iteration 3 ##
No one scale Rater!


No item & user has less than 10 ratinngs!


Data is cleaned!


In [11]:
tv_comedy_anime_core.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1332908 entries, 304 to 7813734
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype
---  ------    --------------    -----
 0   user_id   1332908 non-null  int64
 1   anime_id  1332908 non-null  int64
 2   rating    1332908 non-null  int64
dtypes: int64(3)
memory usage: 40.7 MB


In [12]:
## print summary of  tv & comedy based Anime Rating Data after cleaning ## \n')
print(f'Number of observations: {len(tv_comedy_anime_core)} ')


for i in tv_comedy_anime_core.nunique().index:
    print(f'Number of Unique {i.split('_')[0]}s: {tv_comedy_anime_core.nunique()[i]}')

print(f'*rating scales: {np.sort(tv_comedy_anime_core['rating'].unique())}')

print(f'Density: {len(tv_comedy_anime_core)/(tv_comedy_anime_core.nunique()[0]*tv_comedy_anime_core.nunique().iloc[1])}')

print(tv_comedy_anime_core.head())


Number of observations: 1332908 
Number of Unique users: 32178
Number of Unique animes: 798
Number of Unique ratings: 10
*rating scales: [ 1  2  3  4  5  6  7  8  9 10]
Density: 0.05190847091001168
     user_id  anime_id  rating
304        5        17       6
308        5        24       1
312        5        57       7
313        5        63       1
324        5       170       6


C:\Users\diyas\AppData\Local\Temp\ipykernel_21708\2184630286.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'Density: {len(tv_comedy_anime_core)/(tv_comedy_anime_core.nunique()[0]*tv_comedy_anime_core.nunique().iloc[1])}')


In [13]:
#write to csv
tv_comedy_anime_core.to_csv('core_tv_comedy_rating.csv', index=False)

In [14]:
#splitting dataset into test and train sets using stratification
final_data = pd.read_csv(r'core_tv_comedy_rating.csv')
train,test = train_test_split(final_data,test_size=0.2,random_state=1,stratify=final_data['user_id'])

#train and test csv files
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [16]:
#load final_data,train and test data into surprise objects
reader = Reader(line_format = "user item rating",sep=',',rating_scale=(1,10),skip_lines=1)
WholeSet = Dataset.load_from_file('core_tv_comedy_rating.csv', reader)
S_train = Dataset.load_from_file('train.csv', reader)
S_test = Dataset.load_from_file('test.csv', reader)

In [34]:
#surprise train set
trainset = S_train.build_full_trainset()
WholeSet_train = WholeSet.build_full_trainset()

In [35]:
#surprise test set evaluating RMSE
testset = list(map(lambda x : x[0:3],S_test.raw_ratings))

In [36]:
# testset for evaluating relevence(Precision, Recall, NDCG)
NoRatingSet = WholeSet_train.build_anti_testset(fill=1) #whole unknown rating

_, noRate_test = train_test_split(NoRatingSet, test_size=0.2, random_state=1, stratify=list(map(lambda x: x[0], NoRatingSet)))

testPlusUnknown=testset+noRate_test #testset + 20% of unknown rating for evaluating relevence

In [45]:
#d_compare = pd.merge(df_rating,df_anime,on='anime_id',how='outer')
#d_compare.to_csv("data_ccompare.csv",index=False)

In [44]:
#print(data_ccompare.dtypes)


In [43]:
#data_ccompare=pd.read_csv('data_ccompare.csv',index_col=0)
#data_ccompare['episodes'] = pd.to_numeric(data_ccompare['episodes'], errors='coerce')

#data_ccompare.style.highlight_min(data_ccompare.columns[:-1], axis=0).highlight_max(['Density'], axis=0)


In [46]:
#summary of data that will be used in surprise

print('## Final form of Anime (Comedy & TV based) rating data ##\n')
print(f'Number of Observation: {len(WholeSet.raw_ratings)}')
print(f'Number of Users: {WholeSet_train.n_users}')
print(f'Number of items: {WholeSet_train.n_items}')
print(f'Density: {WholeSet_train.n_ratings/(WholeSet_train.n_users*WholeSet_train.n_items)}')


print('## Surprise trainset  object (80% of whole data) ##\n')
print(f'Object Type: {type(trainset)}'.format())
print(f'Number of observations: {len(S_train.raw_ratings)}')
print(f'Number of Users ( trainset.n_users ): {trainset.n_users}')
print(f'Number of items ( trainset.n_items ): {trainset.n_items}')

print('## Holdeout Testset (20% of whole data) for evalauting Rating Prediction performance##\n')

print(f'Object Type: {type(testset)}'.format())
print(f'Number of Observation: {len(S_test.raw_ratings)}')
print(f'Number of Users: {len(np.unique(np.array(testset)[:,0],return_counts=True)[1])}')
print(f'Number of items: {len(np.unique(np.array(testset)[:,1],return_counts=True)[1])}')

## Final form of Anime (Comedy & TV based) rating data ##

Number of Observation: 1332908
Number of Users: 32178
Number of items: 798
Density: 0.05190847091001168
## Surprise trainset  object (80% of whole data) ##

Object Type: <class 'surprise.trainset.Trainset'>
Number of observations: 1066326
Number of Users ( trainset.n_users ): 32178
Number of items ( trainset.n_items ): 798
## Holdeout Testset (20% of whole data) for evalauting Rating Prediction performance##

Object Type: <class 'list'>
Number of Observation: 266582
Number of Users: 32178
Number of items: 797
